# Challenge 3

In this challenge we will work on the `Orders.csv` data set in the previous [Subsetting and Descriptive Stats lab](../../lab-subsetting-and-descriptive-stats/your-code/main.ipynb). In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

# Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [4]:
# import required libraries
import numpy as np
import pandas as pd

Next, import `Orders.csv` from the "subsetting" lab folder into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [10]:
# enter your code here
orders = pd.read_csv("Orders.csv")
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [20]:
# your code here
orders.info()

orders.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397924 entries, 0 to 397923
Data columns (total 14 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Unnamed: 0    397924 non-null  int64  
 1   InvoiceNo     397924 non-null  int64  
 2   StockCode     397924 non-null  object 
 3   year          397924 non-null  int64  
 4   month         397924 non-null  int64  
 5   day           397924 non-null  int64  
 6   hour          397924 non-null  int64  
 7   Description   397924 non-null  object 
 8   Quantity      397924 non-null  int64  
 9   InvoiceDate   397924 non-null  object 
 10  UnitPrice     397924 non-null  float64
 11  CustomerID    397924 non-null  int64  
 12  Country       397924 non-null  object 
 13  amount_spent  397924 non-null  float64
dtypes: float64(2), int64(8), object(4)
memory usage: 42.5+ MB


Unnamed: 0      0
InvoiceNo       0
StockCode       0
year            0
month           0
day             0
hour            0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID      0
Country         0
amount_spent    0
dtype: int64

In [57]:
a=orders.groupby(['InvoiceNo','CustomerID','Country']).sum()
a

,,,Unnamed: 0,year,month,day,hour,Quantity,UnitPrice,amount_spent
InvoiceNo,CustomerID,Country,,,,,,,,
536365,17850,United Kingdom,21,14070,84,21,56,40,27.37,139.12
536366,17850,United Kingdom,15,4020,24,6,16,12,3.70,22.20
536367,13047,United Kingdom,174,24120,144,36,96,83,58.24,278.73
536368,13047,United Kingdom,90,8040,48,12,32,15,19.10,70.05
536369,13047,United Kingdom,25,2010,12,3,8,3,5.95,17.85
...,...,...,...,...,...,...,...,...,...,...
581583,13777,United Kingdom,1083731,4022,24,10,24,76,3.30,124.60
581584,13777,United Kingdom,1083735,4022,24,10,24,120,2.57,140.64
581585,15804,United Kingdom,11379459,42231,252,105,252,278,37.78,329.05


In [58]:
b=a.groupby(['CustomerID','Country']).mean()


In [59]:
b['amount_spent'].describe()

count     4347.000000
mean       419.163862
std       1794.727171
min          0.000000
25%        178.650000
50%        294.110000
75%        430.342500
max      84236.250000
Name: amount_spent, dtype: float64

In [60]:
b['amount_spent'].quantile(0.95)

938.3539999999997

In [65]:
VIP=b[b.loc[:,'amount_spent'] > 937.18]
VIP

,,Unnamed: 0,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,Country,,,,,,,,
12346,United Kingdom,6.161900e+04,2011.000000,1.000000,2.000000,10.000000,74215.000000,1.040000,77183.600000
12349,Italy,3.544427e+07,146803.000000,803.000000,73.000000,657.000000,631.000000,605.100000,1757.550000
12354,Spain,9.645139e+06,116638.000000,232.000000,232.000000,754.000000,530.000000,261.220000,1079.400000
12357,Switzerland,5.826225e+07,263441.000000,1441.000000,917.000000,2096.000000,2708.000000,438.670000,6207.670000
12359,Cyprus,1.477597e+07,124682.000000,377.000000,195.750000,784.000000,405.500000,534.465000,1593.145000
...,...,...,...,...,...,...,...,...,...
18102,United Kingdom,2.300378e+06,14445.383333,62.433333,21.016667,93.116667,1068.733333,32.348667,4327.621667
18139,United Kingdom,1.292420e+07,53291.500000,291.500000,34.833333,348.333333,926.166667,39.995000,1406.390000
18192,United Kingdom,1.650460e+07,90495.000000,405.000000,180.000000,405.000000,339.000000,180.330000,957.970000


In [73]:
Preferred=b[(b.loc[:,'amount_spent'] < 937.18) & (b.loc[:,'amount_spent'] > 430.022)]
Preferred

,,Unnamed: 0,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,Country,,,,,,,,
12347,Iceland,6.063100e+06,52281.571429,197.571429,63.000000,317.000000,351.142857,68.744286,615.714286
12348,Finland,7.017800e+05,15581.000000,64.250000,27.750000,118.000000,585.250000,44.677500,449.310000
12355,Bahrain,2.354898e+06,26143.000000,65.000000,13.000000,169.000000,240.000000,54.650000,459.400000
12356,Portugal,2.109296e+06,39549.666667,47.333333,61.666667,197.333333,530.333333,62.956667,937.143333
12358,Austria,3.437820e+06,19104.500000,84.000000,26.000000,95.000000,124.000000,78.605000,584.030000
...,...,...,...,...,...,...,...,...,...
18242,United Kingdom,6.031276e+06,39214.500000,149.500000,64.750000,251.250000,254.250000,63.957500,558.122500
18259,United Kingdom,4.630150e+06,28151.666667,142.333333,44.333333,184.333333,238.000000,45.633333,779.533333
18260,United Kingdom,3.008486e+06,44910.500000,96.166667,72.666667,269.333333,246.333333,78.323333,440.533333


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

# Q2: How to identify which country has the most VIP Customers?

# Q3: How to identify which country has the most VIP+Preferred Customers combined?

Provide your solution for Q2 below:

In [69]:
# your code here
VIP.reset_index().Country.mode()

0    United Kingdom
dtype: object

In [76]:
Preferred.reset_index()

combined=pd.concat([VIP, Preferred])
combined

,,Unnamed: 0,year,month,day,hour,Quantity,UnitPrice,amount_spent
CustomerID,Country,,,,,,,,
12346,United Kingdom,6.161900e+04,2011.000000,1.000000,2.000000,10.000000,74215.000000,1.040000,77183.600000
12349,Italy,3.544427e+07,146803.000000,803.000000,73.000000,657.000000,631.000000,605.100000,1757.550000
12354,Spain,9.645139e+06,116638.000000,232.000000,232.000000,754.000000,530.000000,261.220000,1079.400000
12357,Switzerland,5.826225e+07,263441.000000,1441.000000,917.000000,2096.000000,2708.000000,438.670000,6207.670000
12359,Cyprus,1.477597e+07,124682.000000,377.000000,195.750000,784.000000,405.500000,534.465000,1593.145000
...,...,...,...,...,...,...,...,...,...
18242,United Kingdom,6.031276e+06,39214.500000,149.500000,64.750000,251.250000,254.250000,63.957500,558.122500
18259,United Kingdom,4.630150e+06,28151.666667,142.333333,44.333333,184.333333,238.000000,45.633333,779.533333
18260,United Kingdom,3.008486e+06,44910.500000,96.166667,72.666667,269.333333,246.333333,78.323333,440.533333


In [78]:
combined.reset_index().Country.mode()

0    United Kingdom
dtype: object